In [5]:
import string

In [6]:
"""
We want to extract a canonical instrument name from the title.
What are some characteristics of musical instrument names?
"""
open("subjects.txt", "r").readlines()

['!988 Fender Deluxe American Stratocaster (Graffitti\n',
 '"Mason & Hamlin" model "A", 1922 baby grand - $15999\n',
 '"New" Gretsch g5622t - $799 (Brooklyn)\n',
 '"On-Stage " Studio Boom Mic Stand\n',
 '"RickOBacker" Bass  79/ 4001 geddy lee,.first 475 takes it\n',
 '"Rose Tele" Coil taped Seymour Duncan Loaded Telecaster\n',
 '"THE STEW" - (QUEENS)\n',
 '$100 Casio CDP-130 Keyboard - $100 (Greenpoint)\n',
 '$350 OBO Gallien Krueger 115RBH 15" bass cabinet Eminence\n',
 '(2) Gemini XL-500 ii Turntables - $300\n',
 '(LIKE NEW) Gibson Les Paul Junior Doublecut 2019 - Worn\n',
 '(NOT FOR EVERYONE - READ FIRST) Home Audio Production &\n',
 '(READ FIRST) Home Audio Production & Mixing Setup - $1690\n',
 ') )   DW pedal   ) ) - $39 (RIDGEWOOD / BUSHWICK)\n',
 '** Cigar Box Guitars - Hand Made ** - $135 (USA)\n',
 '** NAMM BADGES HERE ** HURRY ALMOST OUT!! 1/24-1/27\n',
 '*** RH Renkus Heinz 2-Way Powered Loudspeakers, 500 watts\n',
 '***2 Traktor Kontrol X1s for sale. Mint condition**** -\n

In [7]:
"""
We can tag the words in the subject by:

* Type (what kind of instrument is it?)
* Attribute (what distinguishes the instrument within its type? e.g: material, use case)
* Brand (who manufactures it?)
* Model (which product is it?)
* Variant (what are the variable properties of the instrument?)
  * Date of manufacture
  * Color
  * Size
  * Condition
  * Quantity e.g in speakers

We can exploit the fact that models frequently imply brands e.g:

* Push 2 -> Ableton
* MPC -> Akai

However we should be careful with this as there could be overlap
between model names, particularly short abbreviated names.

(add example)

Do brands imply type? Not absolutely however perhaps with
some frequency on real data (e.g if someone is selling something
by Dean its most likely to be a Guitar, not a Ukulele).

Thinking out loud about some more categories:

* Series, like Roland MC-* - usually part of the Model

We note that there are a few classes of sentences:

* Sale of a model of item e.g P90 Telecaster
* Sales of a type of item e.g Electric Weighted Keyboard
* Sale of a lot of items with variable type e.g Studio Blowout
* Sale of a lot of items with a fixed type e.g Guitar Pedals
* Things that don't belong in msa (services, items wanted)
"""

"\nWe can tag the words in the subject by:\n\n* Type (what kind of instrument is it?)\n* Attribute (what distinguishes the instrument within its type? e.g: material, use case)\n* Brand (who manufactures it?)\n* Model (which product is it?)\n* Variant (what are the variable properties of the instrument?)\n  * Date of manufacture\n  * Color\n  * Size\n  * Condition\n  * Quantity e.g in speakers\n\nWe can exploit the fact that models frequently imply brands e.g:\n\n* Push 2 -> Ableton\n* MPC -> Akai\n\nHowever we should be careful with this as there could be overlap\nbetween model names, particularly short abbreviated names.\n\n(add example)\n\nDo brands imply type? Not absolutely however perhaps with\nsome frequency on real data (e.g if someone is selling something\nby Dean its most likely to be a Guitar, not a Ukulele).\n\nThinking out loud about some more categories:\n\n* Series, like Roland MC-* - usually part of the Model\n\nWe note that there are a few classes of sentences:\n\n* Sal

In [8]:
"""
Let's manually tag a random sample of 1000 with TYPE, BRAND, MODEL & VARIANT.
gshuf < subjects.txt | head -n 1000 > training-subjects.txt
"""
training_subjects = open("training-subjects.txt", "r").readlines()

def clean_subject(subject):
    # Remove price / location info if present
    # XXX There is a bug here if the subject contains " - " and no price / location
    SEPARATOR = " - "
    if SEPARATOR in subject:
        subject = " ".join(subject.split(SEPARATOR)[:-1])
        
    # Replace special characters with whitespace
    subject = subject.translate(str.maketrans(dict(zip(string.punctuation, " " * len(string.punctuation)))))
    
    # Remove redundant whitespace
    subject = " ".join(subject.split())
    
    return f"{subject}\n"
        
clean_training_subjects = [
    clean_subject(subject)
    for subject in training_subjects
]

with open("clean-training-subjects.txt", "w") as outfile:
    outfile.writelines(clean_training_subjects)

In [9]:
"""
What are some properties that we can intuitively infer from the tagged corpora?

* BRAND tends to be the first word in the subject
* The word "condition" tends to come after the value of a VARIANT
* The word "version" tends to come before the value of a VARIANT
* TYPEs can play as ATTRIBUTEs e.g Rare/ATTRIBUTE Hiwatt/BRAND Pre_1/MODEL Bass_Guitar/ATTRIBUTE Pre_Amp/TYPE
* The word "vintage" can play as a VARIANT or ATTRIBUTE - it is really useless for classification tho
* Tricky to distinguish "thing with case" from "thing case" e.g Srunal/BRAND 260/MODEL 4_4/ATTRIBUTE Beginner/ATTRIBUTE violin_case/TYPE
* The presence of multiple brands or types in a subject is a good indicator of a lot

What are some questions we can ask when we have a large set of tags?

Candidate stop words:

* for sale | f s
* more
* utf 8 ... (this is some auto poster or error)

Candidate acronym expansions:

* MIJ -> Made In Japan
* Mic -> Microphone
* Pre -> Pre-Amplifier
* Synth -> Synthesizer
* Amp -> Amplifier
* Sub -> Subwoofer

Candidate condition compressions:

* New In Box -> New
* Unused -> New
"""

'\nWhat are some properties that we can intuitively infer from the tagged corpora?\n\n* BRAND tends to be the first word in the subject\n* The word "condition" tends to come after the value of a VARIANT\n* The word "version" tends to come before the value of a VARIANT\n* TYPEs can play as ATTRIBUTEs e.g Rare/ATTRIBUTE Hiwatt/BRAND Pre_1/MODEL Bass_Guitar/ATTRIBUTE Pre_Amp/TYPE\n* The word "vintage" can play as a VARIANT or ATTRIBUTE - it is really useless for classification tho\n* Tricky to distinguish "thing with case" from "thing case" e.g Srunal/BRAND 260/MODEL 4_4/ATTRIBUTE Beginner/ATTRIBUTE violin_case/TYPE\n* The presence of multiple brands or types in a subject is a good indicator of a lot\n\nWhat are some questions we can ask when we have a large set of tags?\n\nCandidate stop words:\n\n* for sale | f s\n* more\n* utf 8 ... (this is some auto poster or error)\n\nCandidate acronym expansions:\n\n* MIJ -> Made In Japan\n* Mic -> Microphone\n* Pre -> Pre-Amplifier\n* Synth -> Syn

## Model proposal

We have a prior belief that brand -> model and model -> type are strong relationships and brand -> type is a less strong one.

Given this structure, I would call brand conditionally independent of type (given model) and allow the brand->type relationship to be modeled implicitly.

We can model this as a PGM. ($X = text$, $M = model$, $B = brand$, $T = type$)

Adjacency list:
- $X\to B$
- $X\to M$
- $X\to T$
- $B\to M$
- $M\to T$


We will use a POS-style model to get our text-conditional probabilities for each desired attribute. Each word will recieve a part-of-speech distribution (part of speech here being BRAND, VARIENT, etc rather than VERB, NOUN, etc).

We are likely to classify lexically - running a standard POS tagger and then 

We get $P(B|X)$, $P(M|X)$, and $P(T|X)$ from our POS model.
What else do we need to classify?

Lemma based on our independence assumptions:

$P(T|M,X) = p(T|M,X,B)$

This gives us
$P(B,M,T|X) 
    = P(T|M,B,X) \cdot P(B,M|X)
    = P(T|M,X) \cdot P(M|B,X) \cdot P(B|X)$

First run: naive. We assume that we can further factorize:

$P(B,M,T|X) = P(T|M,X) \cdot P(M|B,X) \cdot P(B|X)$

$P(B,M,T|X) = P(T|M)P(T|X) \cdot P(M|B)P(M|X) \cdot P(B|X)$




Given that our POS model will not include much signal from non-classified words, we are unlikely to find interactions between brand/text w.r.t. model, and model/text w.r.t type. If the structure of $P(M|X)$ was different, this assumption would be less valid. For examples, the word "connector" might be a signal that our type was an accessory, rather than a base instrument. 

We now just need P(T|M) and P(M|B). We will adaptively fit these distributions to data (MAP estimation). Each will be a categorical distribution with smoothing. The type of smoothing will depend on the data - we will have to see if instrument occurances on craigslist are zipf-regular?